In [3]:
# Final Model Total params: 125,780,993
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class UNet(nn.Module):
    def __init__(self, n_classes):
        super(UNet, self).__init__()
        self.enc1 = self.contracting_block(3, 64)
        self.enc2 = self.contracting_block(64, 128)
        self.enc3 = self.contracting_block(128, 256)
        self.enc4 = self.contracting_block(256, 512)
        self.enc5 = self.contracting_block(512, 1024)
        self.enc6 = self.contracting_block(1024, 2048)  # Added an additional layer

        self.upconv5 = self.expansive_block(2048 + 1024, 1024)
        self.upconv4 = self.expansive_block(1024 + 512, 512)
        self.upconv3 = self.expansive_block(512 + 256, 256)
        self.upconv2 = self.expansive_block(256 + 128, 128)
        self.upconv1 = self.expansive_block(128 + 64, 64)

        self.final = nn.Conv2d(64, n_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% probability

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))
        enc6 = self.enc6(self.pool(enc5))  # Added forward pass for the additional layer

        upconv5 = self.upconv5(self.up(enc6, enc5))
        upconv4 = self.upconv4(self.up(upconv5, enc4))
        upconv3 = self.upconv3(self.up(upconv4, enc3))
        upconv2 = self.upconv2(self.up(upconv3, enc2))
        upconv1 = self.dropout(self.upconv1(self.up(upconv2, enc1)))  # Apply dropout before final output

        return torch.sigmoid(self.final(upconv1))

    def contracting_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def expansive_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def up(self, x, bridge):
        up = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        return torch.cat([up, bridge], 1)

    def pool(self, x):
        return F.max_pool2d(x, kernel_size=2, stride=2)

# Instantiate the model and print the summary
model = UNet(n_classes=1)
summary(model, input_size=(3, 256, 256), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
            Conv2d-7        [-1, 128, 128, 128]          73,856
       BatchNorm2d-8        [-1, 128, 128, 128]             256
              ReLU-9        [-1, 128, 128, 128]               0
           Conv2d-10        [-1, 128, 128, 128]         147,584
      BatchNorm2d-11        [-1, 128, 128, 128]             256
             ReLU-12        [-1, 128, 128, 128]               0
           Conv2d-13          [-1, 256, 64, 64]         295,168
      BatchNorm2d-14          [-1, 256,

In [2]:
# second model Total params: 125,756,929
class UNet(nn.Module):
    def __init__(self, n_classes):
        super(UNet, self).__init__()
        self.enc1 = self.contracting_block(3, 64)
        self.enc2 = self.contracting_block(64, 128)
        self.enc3 = self.contracting_block(128, 256)
        self.enc4 = self.contracting_block(256, 512)
        self.enc5 = self.contracting_block(512, 1024)
        self.enc6 = self.contracting_block(1024, 2048)  # Added an additional layer

        self.upconv5 = self.expansive_block(2048 + 1024, 1024)
        self.upconv4 = self.expansive_block(1024 + 512, 512)
        self.upconv3 = self.expansive_block(512 + 256, 256)
        self.upconv2 = self.expansive_block(256 + 128, 128)
        self.upconv1 = self.expansive_block(128 + 64, 64)

        self.final = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))
        enc6 = self.enc6(self.pool(enc5))  # Added forward pass for the additional layer

        upconv5 = self.upconv5(self.up(enc6, enc5))
        upconv4 = self.upconv4(self.up(upconv5, enc4))
        upconv3 = self.upconv3(self.up(upconv4, enc3))
        upconv2 = self.upconv2(self.up(upconv3, enc2))
        upconv1 = self.upconv1(self.up(upconv2, enc1))

        return torch.sigmoid(self.final(upconv1))

    def contracting_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU()
        )

    def expansive_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU()
        )

    def up(self, x, bridge):
        up = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        return torch.cat([up, bridge], 1)

    def pool(self, x):
        return F.max_pool2d(x, kernel_size=2, stride=2)

# Instantiate the model and print the summary
model = UNet(n_classes=1)
summary(model, input_size=(3, 256, 256), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
              ReLU-6        [-1, 128, 128, 128]               0
            Conv2d-7        [-1, 128, 128, 128]         147,584
              ReLU-8        [-1, 128, 128, 128]               0
            Conv2d-9          [-1, 256, 64, 64]         295,168
             ReLU-10          [-1, 256, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         590,080
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 512, 32, 32]       1,180,160
             ReLU-14          [-1, 512,

In [1]:
# First model Total params: 31,378,945 accuracy about 58%
class UNet(nn.Module):
    def __init__(self, n_classes):
        super(UNet, self).__init__()
        self.enc1 = self.contracting_block(3, 64)
        self.enc2 = self.contracting_block(64, 128)
        self.enc3 = self.contracting_block(128, 256)
        self.enc4 = self.contracting_block(256, 512)
        self.enc5 = self.contracting_block(512, 1024)

        self.upconv4 = self.expansive_block(1024 + 512, 512)
        self.upconv3 = self.expansive_block(512 + 256, 256)
        self.upconv2 = self.expansive_block(256 + 128, 128)
        self.upconv1 = self.expansive_block(128 + 64, 64)

        self.final = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))

        upconv4 = self.upconv4(self.up(enc5, enc4))
        upconv3 = self.upconv3(self.up(upconv4, enc3))
        upconv2 = self.upconv2(self.up(upconv3, enc2))
        upconv1 = self.upconv1(self.up(upconv2, enc1))

        return torch.sigmoid(self.final(upconv1))

    def contracting_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU()
        )

    def expansive_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU()
        )

    def up(self, x, bridge):
        up = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        return torch.cat([up, bridge], 1)

    def pool(self, x):
        return F.max_pool2d(x, kernel_size=2, stride=2)

# Instantiate the model and print the summary
model = UNet(n_classes=1)
summary(model, input_size=(3, 256, 256), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
              ReLU-6        [-1, 128, 128, 128]               0
            Conv2d-7        [-1, 128, 128, 128]         147,584
              ReLU-8        [-1, 128, 128, 128]               0
            Conv2d-9          [-1, 256, 64, 64]         295,168
             ReLU-10          [-1, 256, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         590,080
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 512, 32, 32]       1,180,160
             ReLU-14          [-1, 512,